<div style='float: right'><img src='pic/fillmat.png'/></div>
## <div id='fillmat' />フィルマット

In [ ]:
from itertools import product
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
4....
2.1..
..3.3
2....
...31""".split()
nw, nh = len(data[0]), len(data)

### 問題
* 点線の上にタテヨコに線を引いて、盤面をいくつかの畳（幅１マスで長さ１～４マスの四角形）に区切ります
* 数字は、その数字を含む畳の面積を、１マスを１として表します
* 数字の入らない畳を作ってもよいが、２つ以上の数字を含む畳を作ってはいけません
* 同じ面積の畳をタテヨコに隣り合わせてはいけません
* 畳の境界線が十字に交差してはいけません

### 変数
* 数字ごとの候補 (1)

### 制約
* 1つの候補を選ぶ (2)
* 4つの角を接する組合せの合計が≦3 (3)
* 同じ面積の候補同士で隣り合うものの合計≦1 (4)

In [ ]:
m = LpProblem()
vls = [] # list of (var, pos_list)
cs = [[LpAffineExpression() for j in range(nw)] for i in range(nh)] # cons of pos
dic = {} # key:(y_start, x_start, y_len, x_len), value:(var, pos_list)
def chk(v, ky):
    global m
    if ky in dic:
        m += v + dic[ky][0] <= 1 # (4)
def cand(i, j, n, dy, dx):
    p, q = [], []
    for k in range(n):
        y, x = i + k * dy, j + k * dx
        p.append((y,x))
        if data[y][x].isdigit():
            q.append(int(data[y][x]))
    if len(q) >= 2 or (len(q) == 1 and q[0] != n):
        return
    v = addbinvar() # (1)
    for k in range(max(1, n * dx)):
        chk(v, (i - 1, j + k - n + 1, 0, n))
        chk(v, (i - n, j + k, n, 0))
    for k in range(max(1, n * dy)):
        chk(v, (i + k, j - n, 0, n))
        chk(v, (i + k - n + 1, j - 1, n, 0))
    vls.append((v, p))
    dic[(i, j, dy*n, dx*n)] = vls[-1]
for i, j, k in product(range(nh), range(nw), range(4)):
    if i + k < nh:
        cand(i, j, k+1, 1, 0)
    if k > 0 and j+k < nw:
        cand(i, j, k+1, 0, 1)
for i, vl in enumerate(vls):
    for y, x in vl[1]:
        cs[y][x] += vl[0]
def chk2(ky1, ky2, ky3, ky4):
    global m
    if ky1 in dic and ky2 in dic and ky3 in dic and ky4 in dic:
        m += dic[ky1][0] + dic[ky2][0] + dic[ky3][0] + dic[ky4][0] <= 3 # (3)
for i, j, k in product(range(nh), range(nw), range(4)):
    m += cs[i][j] == 1 # (2)
    if i == 0 or j == 0:
        continue
    for k1 in range(1,9):
        y1, x1, z1, w1 = (i - k1, j - 1, k1, 0) if k1 < 5 else (i - 1, j - k1 + 4, 0, k1 - 4)
        if y1 < 0 or x1 < 0:
            continue
        for k2 in range(1,9):
            y2, x2, z2, w2 = (i, j - 1, k2, 0) if k2 < 5 else (i - 1, j - k2 + 4, 0, k2 - 4)
            if y2 + z2 > nh or x2 < 0:
                continue
            for k3 in range(1,9):
                y3, x3, z3, w3 = (i, j, k3, 0) if k3 < 5 else (i, j, 0, k3 - 4)
                if y3 + z3 > nh or x3 + w3 > nw:
                    continue
                for k4 in range(1,9):
                    y4, x4, z4, w4 = (i - k4, j, k4, 0) if k4 < 5 else (i - 1, j, 0, k4 - 4)
                    if y4 < 0 or x4 + w4 > nw:
                        continue
                    chk2((y1, x1, z1, w1), (y2, x2, z2, w2), (y3, x3, z3, w3), (y4, x4, z4, w4))
%time m.solve()
i, ss = 65, [[0]*nw for i in range(nh)]
for vl in vls:
    if value(vl[0]) > 0.5:
        for y, x in vl[1]:
            ss[y][x] = chr(i)
        i += 1
print('\n'.join(' '.join(s) for s in ss))